## Loading the database

In [3]:
import sqlite3
db_connection = sqlite3.connect('../sqlite/db/checks.db')
db_cursor = db_connection.cursor()

### Getting information from timestamps

In [5]:
# Getting all lines
db_cursor.execute('''SELECT * FROM timestamps''')
for line in db_cursor.fetchall():
    print(line)

('ASR903_5', 'isis', 'before', '2020-11-17 11:47:12')
('ASR903_5', 'isis', 'before', '2020-11-17 11:51:45')
('ASR903_5', 'route_summary', 'before', '2020-11-17 14:50:26')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:08:36')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:09:15')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:09:19')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:10:10')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:10:13')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:11:04')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:12:03')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:12:15')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:27:33')
('ASR903_5', 'route_summary', 'before', '2020-11-17 15:30:10')
('ASR903_5', 'routes', 'before', '2020-11-17 16:57:38')
('ASR903_5', 'routes', 'before', '2020-11-17 16:57:56')
('ASR903_5', 'routes', 'before', '2020-11-17 16:58:35')
('ASR903_5', 'routes', 'before'

In [9]:
# Getting the lines for specific TestName and WhenTested = before
db_cursor.execute('''SELECT * FROM timestamps WHERE TestName = "isis" AND WhenTested = "before"''')
for line in db_cursor.fetchall():
    print(line)

('ASR903_5', 'isis', 'before', '2020-11-17 11:47:12')
('ASR903_5', 'isis', 'before', '2020-11-17 11:51:45')


In [23]:
my_router = 'ASR903_5'
filter_tuple = []
# the `,` avoids that the .extend() iterates through the string
# makes filter_tuple = ['A', 'S', 'R' ...]
filter_tuple.extend((my_router,))

# Getting the lines for specific TestName and WhenTested = before
# Only the oldest
# fetchone() is enough as I only have one entry (I should...)
db_cursor.execute('''   SELECT * FROM timestamps 
                        WHERE TestName = "isis" AND WhenTested = "before" AND MyRouter = (?)
                        ORDER BY Timestamp DESC
                        LIMIT 1''', filter_tuple)

timestamp_test_before_tuple = db_cursor.fetchone()

print(timestamp_test_before_tuple)

('ASR903_5', 'isis', 'before', '2020-11-17 11:51:45')


In [32]:
# filter_tuple is only for data, not for table names
# https://stackoverflow.com/questions/51312952/python-sqlite3-using-f-strings-for-update-database-function

filter_tuple = []
my_router = timestamp_test_before_tuple[0]
table_name = timestamp_test_before_tuple[1]
timestamp = timestamp_test_before_tuple[3]

filter_tuple.extend((my_router, timestamp))

db_cursor.execute(f'''  SELECT * FROM {table_name} 
                        WHERE MyRouter = (?) AND Timestamp = (?)''', filter_tuple)

isis_before = db_cursor.fetchall()
for line in isis_before:
    print(line)

('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3', '2020-11-17 11:51:45')
('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4', '2020-11-17 11:51:45')


In [42]:
isis_after = [{'MyRouter': 'ASR903_5', 'Neighbor': 'ASR903_2', 'Type': 'L2', 'Interface': 'BD1', 'Ip': '100.100.35.2', 'Timestamp': '2020-11-17 11:51:45'},{'MyRouter': 'ASR903_5', 'Neighbor': 'ASR903_3', 'Type': 'L2', 'Interface': 'Te0/2/4', 'Ip': '100.100.43.4', 'Timestamp': '2020-11-17 11:51:45'}]

for line in isis_after:
    print(line)

{'MyRouter': 'ASR903_5', 'Neighbor': 'ASR903_2', 'Type': 'L2', 'Interface': 'BD1', 'Ip': '100.100.35.2', 'Timestamp': '2020-11-17 11:51:45'}
{'MyRouter': 'ASR903_5', 'Neighbor': 'ASR903_3', 'Type': 'L2', 'Interface': 'Te0/2/4', 'Ip': '100.100.43.4', 'Timestamp': '2020-11-17 11:51:45'}


In [40]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

db_connection.row_factory = dict_factory
db_cursor = db_connection.cursor()

db_cursor.execute(f'''  SELECT * FROM {table_name} 
                        WHERE MyRouter = (?) AND Timestamp = (?)''', filter_tuple)

isis_before = db_cursor.fetchall()
for line in isis_before:
    print(line)





{'MyRouter': 'ASR903_5', 'Neighbor': 'ASR903_3', 'Type': 'L2', 'Interface': 'BD1', 'Ip': '100.100.35.3', 'Timestamp': '2020-11-17 11:51:45'}
{'MyRouter': 'ASR903_5', 'Neighbor': 'ASR903_4', 'Type': 'L2', 'Interface': 'Te0/2/4', 'Ip': '100.100.45.4', 'Timestamp': '2020-11-17 11:51:45'}


## Getting the information for the routes

In [54]:
my_router = 'ASR903_5'
filter_tuple = []
# the `,` avoids that the .extend() iterates through the string
# makes filter_tuple = ['A', 'S', 'R' ...]
filter_tuple.extend((my_router,))

# Getting the lines for specific TestName and WhenTested = before
# Only the oldest
# fetchone() is enough as I only have one entry (I should...)
db_cursor.execute('''   SELECT * FROM timestamps 
                        WHERE TestName = "routes" AND WhenTested = "before" AND MyRouter = (?)
                        ORDER BY Timestamp DESC
                        LIMIT 1''', filter_tuple)

# timestamp_test_before_tuple is the oldest line in the DB, for the specified (MyRouter, TestName)
timestamp_test_before_tuple = db_cursor.fetchone()

# filter_tuple is only for data, not for table names
# https://stackoverflow.com/questions/51312952/python-sqlite3-using-f-strings-for-update-database-function

filter_tuple = []
my_router = timestamp_test_before_tuple['MyRouter']
table_name = timestamp_test_before_tuple['TestName']
timestamp = timestamp_test_before_tuple['Timestamp']

filter_tuple.extend((my_router, timestamp))

db_cursor.execute(f'''  SELECT * FROM {table_name} 
                        WHERE MyRouter = (?) AND Timestamp = (?)''', filter_tuple)

routes_before = db_cursor.fetchall()


In [56]:
routes_after = [{'MyRouter': 'ASR903_5', 'TestName': 'routes', 'WhenTested': 'before', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'connected', 'Route': '10.67.23.25/32', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'isis', 'Route': '10.67.120.250/32', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'isis', 'Route': '10.75.1.50/32', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'isis', 'Route': '10.75.5.50/32', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'isis', 'Route': '100.1.0.0/16', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'isis', 'Route': '100.2.0.0/16', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'isis', 'Route': '100.4.0.0/16', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'connected', 'Route': '100.5.0.0/16', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'connected', 'Route': '100.100.35.0/24', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'local', 'Route': '100.100.35.5/32', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'connected', 'Route': '100.100.45.0/24', 'Timestamp': '2020-11-17 17:00:48'}, {'MyRouter': 'ASR903_5', 'VRF': 'default', 'Protocol': 'local', 'Route': '100.100.45.5/32', 'Timestamp': '2020-11-17 17:00:48'}]

## Diff with Genie

In [46]:
from genie.utils.diff import Diff

## isis
dd = Diff(isis_before, isis_after)
dd.findDiff()

print(dd)

 0:
- Ip: 100.100.35.3
+ Ip: 100.100.35.2
- Neighbor: ASR903_3
+ Neighbor: ASR903_2
 1:
- Ip: 100.100.45.4
+ Ip: 100.100.43.4
- Neighbor: ASR903_4
+ Neighbor: ASR903_3


In [57]:
from genie.utils.diff import Diff

## routes
dd = Diff(routes_before, routes_after)
dd.findDiff()

print(dd)

-index[13]: 
- MyRouter: ASR903_5
- Protocol: local
- Route: 100.100.45.5/32
- Timestamp: 2020-11-17 17:00:48
- VRF: default
 0:
- Protocol: connected
- Route: 10.67.23.250/32
+ TestName: routes
- VRF: default
+ WhenTested: before
 1:
- Protocol: isis
+ Protocol: connected
- Route: 10.67.120.250/32
+ Route: 10.67.23.25/32
 2:
- Route: 10.68.120.250/32
+ Route: 10.67.120.250/32
 9:
- Protocol: local
+ Protocol: connected
- Route: 100.5.200.200/32
+ Route: 100.100.35.0/24
 10:
- Protocol: connected
+ Protocol: local
- Route: 100.100.35.0/24
+ Route: 100.100.35.5/32
 11:
- Protocol: local
+ Protocol: connected
- Route: 100.100.35.5/32
+ Route: 100.100.45.0/24
 12:
- Protocol: connected
+ Protocol: local
- Route: 100.100.45.0/24
+ Route: 100.100.45.5/32


## Testing with real output for the routes

In [63]:
test_routes_before = { 'vrf': { 'default': { 'address_family': { 'ipv4': { 'routes': { '10.67.120.250/32': { 'active': True,
                                                                                        'metric': 1000,
                                                                                        'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                              'next_hop': '100.100.35.3',
                                                                                                                              'outgoing_interface': 'BDI1',
                                                                                                                              'updated': '00:08:50'}}},
                                                                                        'route': '10.67.120.250/32',
                                                                                        'route_preference': 115,
                                                                                        'source_protocol': 'isis',
                                                                                        'source_protocol_codes': 'i '
                                                                                                                 'L2'},
                                                                  '10.67.23.250/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'Loopback0': { 'outgoing_interface': 'Loopback0'}}},
                                                                                       'route': '10.67.23.250/32',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '10.68.120.250/32': { 'active': True,
                                                                                        'metric': 1020,
                                                                                        'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                              'next_hop': '100.100.35.3',
                                                                                                                              'outgoing_interface': 'BDI1',
                                                                                                                              'updated': '00:08:50'}}},
                                                                                        'route': '10.68.120.250/32',
                                                                                        'route_preference': 115,
                                                                                        'source_protocol': 'isis',
                                                                                        'source_protocol_codes': 'i '
                                                                                                                 'L2'},
                                                                  '10.75.1.50/32': { 'active': True,
                                                                                     'metric': 1100,
                                                                                     'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                           'next_hop': '100.100.35.3',
                                                                                                                           'outgoing_interface': 'BDI1',
                                                                                                                           'updated': '00:08:50'}}},
                                                                                     'route': '10.75.1.50/32',
                                                                                     'route_preference': 115,
                                                                                     'source_protocol': 'isis',
                                                                                     'source_protocol_codes': 'i '
                                                                                                              'L2'},
                                                                  '10.75.5.50/32': { 'active': True,
                                                                                     'metric': 1110,
                                                                                     'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                           'next_hop': '100.100.35.3',
                                                                                                                           'outgoing_interface': 'BDI1',
                                                                                                                           'updated': '00:08:50'}}},
                                                                                     'route': '10.75.5.50/32',
                                                                                     'route_preference': 115,
                                                                                     'source_protocol': 'isis',
                                                                                     'source_protocol_codes': 'i '
                                                                                                              'L2'},
                                                                  '100.1.0.0/16': { 'active': True,
                                                                                    'metric': 1100,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.1.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.100.35.0/24': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'BDI1': { 'outgoing_interface': 'BDI1'}}},
                                                                                       'route': '100.100.35.0/24',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '100.100.35.5/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'BDI1': { 'outgoing_interface': 'BDI1'}}},
                                                                                       'route': '100.100.35.5/32',
                                                                                       'source_protocol': 'local',
                                                                                       'source_protocol_codes': 'L'},
                                                                  '100.100.45.0/24': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'TenGigabitEthernet0/2/4': { 'outgoing_interface': 'TenGigabitEthernet0/2/4'}}},
                                                                                       'route': '100.100.45.0/24',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '100.100.45.5/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'TenGigabitEthernet0/2/4': { 'outgoing_interface': 'TenGigabitEthernet0/2/4'}}},
                                                                                       'route': '100.100.45.5/32',
                                                                                       'source_protocol': 'local',
                                                                                       'source_protocol_codes': 'L'},
                                                                  '100.2.0.0/16': { 'active': True,
                                                                                    'metric': 1110,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.2.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.4.0.0/16': { 'active': True,
                                                                                    'metric': 1020,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.4.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.5.0.0/16': { 'active': True,
                                                                                    'next_hop': { 'outgoing_interface': { 'BDI100': { 'outgoing_interface': 'BDI100'}}},
                                                                                    'route': '100.5.0.0/16',
                                                                                    'source_protocol': 'connected',
                                                                                    'source_protocol_codes': 'C'},
                                                                  '100.5.200.200/32': { 'active': True,
                                                                                        'next_hop': { 'outgoing_interface': { 'BDI100': { 'outgoing_interface': 'BDI100'}}},
                                                                                        'route': '100.5.200.200/32',
                                                                                        'source_protocol': 'local',
                                                                                        'source_protocol_codes': 'L'}}}}}}}

In [74]:
test_routes_after_modify = { 'vrf': { 'default': { 'address_family': { 'ipv4': { 'routes': { '10.67.120.250/32': { 'active': True,
                                                                                        'metric': 1000,
                                                                                        'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                              'next_hop': '100.100.35.3',
                                                                                                                              'outgoing_interface': 'BDI1',
                                                                                                                              'updated': '00:08:50'}}},
                                                                                        'route': '10.67.120.250/32',
                                                                                        'route_preference': 115,
                                                                                        'source_protocol': 'isis',
                                                                                        'source_protocol_codes': 'i '
                                                                                                                 'L2'},
                                                                  '10.67.23.250/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'Loopback0': { 'outgoing_interface': 'Loopback0'}}},
                                                                                       'route': '10.67.23.250/32',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '10.68.120.250/32': { 'active': True,
                                                                                        'metric': 1020,
                                                                                        'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                              'next_hop': '100.100.35.3',
                                                                                                                              'outgoing_interface': 'BDI1',
                                                                                                                              'updated': '00:08:50'}}},
                                                                                        'route': '10.68.120.250/32',
                                                                                        'route_preference': 115,
                                                                                        'source_protocol': 'isis',
                                                                                        'source_protocol_codes': 'i '
                                                                                                                 'L2'},
                                                                  '10.75.1.50/32': { 'active': True,
                                                                                     'metric': 1100,
                                                                                     'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                           'next_hop': '100.100.35.3',
                                                                                                                           'outgoing_interface': 'BDI1',
                                                                                                                           'updated': '00:08:50'}}},
                                                                                     'route': '10.75.1.50/32',
                                                                                     'route_preference': 115,
                                                                                     'source_protocol': 'isis',
                                                                                     'source_protocol_codes': 'i '
                                                                                                              'L2'},
                                                                  '10.75.5.50/32': { 'active': True,
                                                                                     'metric': 1110,
                                                                                     'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                           'next_hop': '100.100.35.3',
                                                                                                                           'outgoing_interface': 'BDI1',
                                                                                                                           'updated': '00:08:50'}}},
                                                                                     'route': '10.75.5.50/32',
                                                                                     'route_preference': 115,
                                                                                     'source_protocol': 'isis',
                                                                                     'source_protocol_codes': 'i '
                                                                                                              'L2'},
                                                                  '100.1.0.0/16': { 'active': True,
                                                                                    'metric': 1100,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.1.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.100.35.0/24': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'BDI1': { 'outgoing_interface': 'BDI1'}}},
                                                                                       'route': '100.100.35.0/24',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '100.100.35.6/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'BDI1': { 'outgoing_interface': 'BDI1'}}},
                                                                                       'route': '100.100.35.5/32',
                                                                                       'source_protocol': 'local',
                                                                                       'source_protocol_codes': 'L'},
                                                                  '100.100.45.1/24': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'TenGigabitEthernet0/2/4': { 'outgoing_interface': 'TenGigabitEthernet0/2/4'}}},
                                                                                       'route': '100.100.45.0/24',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '100.100.45.6/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'TenGigabitEthernet0/2/4': { 'outgoing_interface': 'TenGigabitEthernet0/2/4'}}},
                                                                                       'route': '100.100.45.5/32',
                                                                                       'source_protocol': 'local',
                                                                                       'source_protocol_codes': 'L'},
                                                                  '100.2.0.0/16': { 'active': True,
                                                                                    'metric': 1110,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.2.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.4.0.0/16': { 'active': True,
                                                                                    'metric': 1020,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.4.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.5.0.0/16': { 'active': True,
                                                                                    'next_hop': { 'outgoing_interface': { 'BDI100': { 'outgoing_interface': 'BDI100'}}},
                                                                                    'route': '100.5.0.0/16',
                                                                                    'source_protocol': 'connected',
                                                                                    'source_protocol_codes': 'C'},
                                                                  '100.5.200.200/32': { 'active': True,
                                                                                        'next_hop': { 'outgoing_interface': { 'BDI100': { 'outgoing_interface': 'BDI100'}}},
                                                                                        'route': '100.5.200.200/32',
                                                                                        'source_protocol': 'local',
                                                                                        'source_protocol_codes': 'L'}}}}}}}

In [72]:
test_routes_after_remove = { 'vrf': { 'default': { 'address_family': { 'ipv4': { 'routes': { '10.67.120.250/32': { 'active': True,
                                                                                        'metric': 1000,
                                                                                        'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                              'next_hop': '100.100.35.3',
                                                                                                                              'outgoing_interface': 'BDI1',
                                                                                                                              'updated': '00:08:50'}}},
                                                                                        'route': '10.67.120.250/32',
                                                                                        'route_preference': 115,
                                                                                        'source_protocol': 'isis',
                                                                                        'source_protocol_codes': 'i '
                                                                                                                 'L2'},
                                                                  '10.67.23.250/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'Loopback0': { 'outgoing_interface': 'Loopback0'}}},
                                                                                       'route': '10.67.23.250/32',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '10.75.1.50/32': { 'active': True,
                                                                                     'metric': 1100,
                                                                                     'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                           'next_hop': '100.100.35.3',
                                                                                                                           'outgoing_interface': 'BDI1',
                                                                                                                           'updated': '00:08:50'}}},
                                                                                     'route': '10.75.1.50/32',
                                                                                     'route_preference': 115,
                                                                                     'source_protocol': 'isis',
                                                                                     'source_protocol_codes': 'i '
                                                                                                              'L2'},
                                                                  '10.75.5.50/32': { 'active': True,
                                                                                     'metric': 1110,
                                                                                     'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                           'next_hop': '100.100.35.3',
                                                                                                                           'outgoing_interface': 'BDI1',
                                                                                                                           'updated': '00:08:50'}}},
                                                                                     'route': '10.75.5.50/32',
                                                                                     'route_preference': 115,
                                                                                     'source_protocol': 'isis',
                                                                                     'source_protocol_codes': 'i '
                                                                                                              'L2'},
                                                                  '100.1.0.0/16': { 'active': True,
                                                                                    'metric': 1100,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.1.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.100.35.0/24': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'BDI1': { 'outgoing_interface': 'BDI1'}}},
                                                                                       'route': '100.100.35.0/24',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '100.100.45.0/24': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'TenGigabitEthernet0/2/4': { 'outgoing_interface': 'TenGigabitEthernet0/2/4'}}},
                                                                                       'route': '100.100.45.0/24',
                                                                                       'source_protocol': 'connected',
                                                                                       'source_protocol_codes': 'C'},
                                                                  '100.100.45.5/32': { 'active': True,
                                                                                       'next_hop': { 'outgoing_interface': { 'TenGigabitEthernet0/2/4': { 'outgoing_interface': 'TenGigabitEthernet0/2/4'}}},
                                                                                       'route': '100.100.45.5/32',
                                                                                       'source_protocol': 'local',
                                                                                       'source_protocol_codes': 'L'},
                                                                  '100.2.0.0/16': { 'active': True,
                                                                                    'metric': 1110,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.2.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.4.0.0/16': { 'active': True,
                                                                                    'metric': 1020,
                                                                                    'next_hop': { 'next_hop_list': { 1: { 'index': 1,
                                                                                                                          'next_hop': '100.100.35.3',
                                                                                                                          'outgoing_interface': 'BDI1',
                                                                                                                          'updated': '00:08:50'}}},
                                                                                    'route': '100.4.0.0/16',
                                                                                    'route_preference': 115,
                                                                                    'source_protocol': 'isis',
                                                                                    'source_protocol_codes': 'i '
                                                                                                             'L2'},
                                                                  '100.5.200.200/32': { 'active': True,
                                                                                        'next_hop': { 'outgoing_interface': { 'BDI100': { 'outgoing_interface': 'BDI100'}}},
                                                                                        'route': '100.5.200.200/32',
                                                                                        'source_protocol': 'local',
                                                                                        'source_protocol_codes': 'L'}}}}}}}

In [75]:
from genie.utils.diff import Diff

## routes -- modified
dd = Diff(test_routes_before, test_routes_after_modify)
dd.findDiff()

print("Routes have been modified but none has been removed")
print(dd)

print()
print()
print()

## routes -- removed
dd = Diff(test_routes_before, test_routes_after_remove)
dd.findDiff()

print("Routes have been removed but none has been modified")
print(dd)

Routes have been modified but none has been removed
 vrf:
  default:
   address_family:
    ipv4:
     routes:
-     100.100.35.5/32: 
-      active: True
-      next_hop: 
-       outgoing_interface: 
-        BDI1: 
-         outgoing_interface: BDI1
-      route: 100.100.35.5/32
-      source_protocol: local
-      source_protocol_codes: L
+     100.100.35.6/32: 
+      active: True
+      next_hop: 
+       outgoing_interface: 
+        BDI1: 
+         outgoing_interface: BDI1
+      route: 100.100.35.5/32
+      source_protocol: local
+      source_protocol_codes: L
-     100.100.45.0/24: 
-      active: True
-      next_hop: 
-       outgoing_interface: 
-        TenGigabitEthernet0/2/4: 
-         outgoing_interface: TenGigabitEthernet0/2/4
-      route: 100.100.45.0/24
-      source_protocol: connected
-      source_protocol_codes: C
+     100.100.45.1/24: 
+      active: True
+      next_hop: 
+       outgoing_interface: 
+        TenGigabitEthernet0/2/4: 
+         outgoing_

In [66]:
a = {'a':5, 'b':7, 'c':8}
b = {'c':8, 'a':5, 'b':7}

dd = Diff(a, b)
dd.findDiff()

print("Same values in a different order")
print(dd)

a = {'a':5, 'b':7, 'c':8}
b = {'b':7, 'c':8}

dd = Diff(a, b)
dd.findDiff()

print("Missing a value")
print(dd)

Same values in a different order

Missing a value
-a: 5


In [ ]:
prin